In [26]:
import os
import torch
import sys
from PIL import Image
import fitz  # PyMuPDF

# Initialize paths
BASE_DIR = "/Users/rithwanali/Documents/auto_blueprint/Auto-Blueprints"
YOLO_DIR = os.path.join("/Users/rithwanali/Documents/auto_blueprint/Auto-Blueprints/yolov5")
IMAGE_PATH = os.path.join(BASE_DIR, "kofferdals_1.10.png")
# PDF_PATH = os.path.join(BASE_DIR, "example.pdf")  # Path to your PDF file
OUTPUT_DIR = "results_yolo"
MODEL_NAME = "yolov5su.pt"

def validate_paths():
    """Validate that all required paths exist"""
    if not os.path.exists(YOLO_DIR):
        raise FileNotFoundError(f"YOLOv5 directory not found at: {YOLO_DIR}")
    if not os.path.exists(IMAGE_PATH):
        raise FileNotFoundError(f"Image not found at: {IMAGE_PATH}")
    # if not os.path.exists(PDF_PATH):
    #     raise FileNotFoundError(f"PDF not found at: {PDF_PATH}")
    if not os.path.exists(os.path.join(YOLO_DIR, MODEL_NAME)):
        raise FileNotFoundError(f"Model not found at: {os.path.join(YOLO_DIR, MODEL_NAME)}")

def extract_data_from_pdf(pdf_path):
    """Extract text and metadata from a PDF using PyMuPDF (fitz)"""
    doc = fitz.open(pdf_path)
    extracted_data = {}

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        extracted_data[f"page_{page_num + 1}"] = text

    return extracted_data

def run_yolo_detection():
    # Validate all paths before proceeding
    validate_paths()
    
    # Add YOLOv5 directory to system path
    if YOLO_DIR not in sys.path:
        sys.path.append(YOLO_DIR)

    try:
        # Import YOLOv5 modules
        from models.experimental import attempt_load
        from utils.general import non_max_suppression
        from utils.torch_utils import select_device
    except ImportError as e:
        print(f"ImportError: {e}")
        print("Ensure the YOLOv5 repository is up-to-date and all dependencies are installed.")
        return
    
    # Select device (CPU/GPU)
    device = select_device('CPU')  # Change to '0' for GPU if available
    
    # Load your fine-tuned model
    model_path = os.path.join(YOLO_DIR, MODEL_NAME)
    print(f"Loading model from: {model_path}")
    model = attempt_load(model_path, device=device)
    
    # Create output directory if it doesn't exist
    output_path = os.path.join(os.path.dirname(IMAGE_PATH), OUTPUT_DIR)
    os.makedirs(output_path, exist_ok=True)
    
    print(f"Processing image: {IMAGE_PATH}")
    process_single_image(IMAGE_PATH, model, output_path, device)

def process_single_image(image_path, model, output_dir, device):
    try:
        # Import here to avoid potential circular imports
        from utils.datasets import letterbox
        from utils.general import check_img_size, scale_coords
        import numpy as np
        
        # Load and preprocess image
        img = Image.open(image_path)
        img = letterbox(np.array(img))[0]  # Resize and pad image
        img = img.transpose(2, 0, 1)  # HWC to CHW
        img = torch.from_numpy(img).to(device)
        img = img.float()
        img /= 255.0  # Normalize pixel values to [0, 1]
        if img.ndimension() == 3:
            img = img.unsqueeze(0)  # Add batch dimension
        
        # Run inference
        pred = model(img)[0]
        
        # Apply Non-Max Suppression (NMS)
        pred = non_max_suppression(pred)
        
        # Process detections
        for i, det in enumerate(pred):
            if len(det):
                # Rescale bounding boxes to original image size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img.shape).round()
                
                # Print results
                print("\nDetections:")
                for *xyxy, conf, cls in det:
                    print(f"Class: {int(cls)}, Confidence: {conf:.2f}, Bbox: {xyxy}")
                
                # Save results with bounding boxes drawn
                import cv2
                im0 = cv2.imread(image_path)
                for *xyxy, conf, cls in det:
                    # Draw bounding boxes
                    cv2.rectangle(im0, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (0, 255, 0), 2)
                
                # Save the output image
                output_path = os.path.join(output_dir, 'result_' + os.path.basename(image_path))
                cv2.imwrite(output_path, im0)
                print(f"Results saved to: {output_path}")
        
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        raise e

if __name__ == "__main__":
    print("Starting YOLOv5 detection...")
    print(f"Base directory: {BASE_DIR}")
    print(f"YOLOv5 directory: {YOLO_DIR}")
    print(f"Image path: {IMAGE_PATH}")
    print(f"PDF path: {PDF_PATH}")
    
    try:
        # Extract data from PDF
        pdf_data = extract_data_from_pdf(PDF_PATH)
        print("\nExtracted PDF Data:")
        for page, text in pdf_data.items():
            print(f"{page}:\n{text}\n")
        
        # Run YOLOv5 detection
        run_yolo_detection()
        print("\nProcessing complete. Results saved to:", os.path.join(os.path.dirname(IMAGE_PATH), OUTPUT_DIR))
    except Exception as e:
        print(f"Error during execution: {str(e)}")

Starting YOLOv5 detection...
Base directory: /Users/rithwanali/Documents/auto_blueprint/Auto-Blueprints
YOLOv5 directory: /Users/rithwanali/Documents/auto_blueprint/Auto-Blueprints/yolov5
Image path: /Users/rithwanali/Documents/auto_blueprint/Auto-Blueprints/kofferdals_1.10.png
PDF path: /Users/rithwanali/Documents/auto_blueprint/Auto-Blueprints/example.pdf
Error during execution: no such file: '/Users/rithwanali/Documents/auto_blueprint/Auto-Blueprints/example.pdf'
